# Using the segmented mirror in HCIPy

Just using the segmented mirror this time directly from the PASTIS module.

The `SegmentedMirror()` works with any segmented aperture from `HCIPy`, provided you put it in a form where the aperture function also returns the segment positions. Currently I only have that for the HiCAT/ATLAST aperture, but it's not hard to make once you have an aperture functions, since the only thing is literally to add the segment positions as a return.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from hcipy import *

os.chdir('../../pastis/')
from config import CONFIG_INI
import util_pastis as util
import atlast_imaging as atim

### Instantiate an SM

In [ ]:
# Parameters
npix = 1024
wvln = 1
lamD = 15
samp = 4
which_tel = CONFIG_INI.get('telescope', 'name')
PUP_DIAMETER = CONFIG_INI.getfloat(which_tel, 'diameter')

In [ ]:
# HCIPy grids and propagator
pupil_grid = make_pupil_grid(dims=1024, diameter=1)
focal_grid = make_focal_grid(pupil_grid, samp, lamD)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
aper, seg_pos = atim.get_atlast_aperture(normalized=True)
aper = evaluate_supersampled(aper, pupil_grid, 2)

imshow_field(aper)
plt.title('Aperture')

In [ ]:
# Instantiate the segmented mirror
sm = atim.SegmentedMirror(aper, seg_pos)

In [ ]:
# Make a pupil plane wavefront from aperture
wf = Wavefront(aper, wavelength=wvln)
imshow_field(wf.intensity)
plt.title('Wavefront intensity')

### Field at SM pupil plane

In [ ]:
# Propagate the wf through the SM and display its intensity
# and phase in that pupil plane, before propagation to image plane.
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('WF prop through SM, still in pupil plane')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Field in image plane after SM

In [ ]:
# Propagate from SM to image plane and display intensity and phase
im_ref = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im_ref.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Aberrate some segments in tip and tilt

In [ ]:
for i in [2, 34, 3, 19, 23,]:
    sm.set_segment(i, 0, 2, 2)
print(sm.coef)

In [ ]:
# SM in pupil plane
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane with tilts')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Image after SM with tilts
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM with tilts')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Flatten the SM and put pistons on instead

In [ ]:
sm.flatten()

In [ ]:
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane - flattened')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

The wavefront phase is displayed in units of radians.

In [ ]:
#for i in [2, 34, 3, 19, 23, 28, 14]:
for i in [19, 28]:
    sm.set_segment(i, 0.1, 0, 0)   # This input is in meters, the phasemap below is in radians though.

In [ ]:
# Pupil plane SM image
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('SM in pupil plane - with pistons')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Image after SM with pistons
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM with pistons')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Display piston aberrated and reference image next to each other
plt.figure(figsize=(18, 9))
plt.suptitle('Reference image and piston image')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref.intensity))
plt.title('Reference image')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(np.log10(im.intensity))
plt.title('Image with piston aberrations')
plt.colorbar()